# Proximity analysis
In this notebook we will be working with data from [OpenStreetMap](https://openstreetmap.org) to do proximity analysis to various urban amenities.

In [ ]:
import geopandas
import pyproj
from shapely.ops import transform

%matplotlib inline

## Import data

In [ ]:
food_sources_file = "data/Tampere-food_sources.geojson"
bus_stops_file = "data/Tampere-bus_stops.gpkg"
building_footprints_file = "data/Tampere-buildings.gpkg"

In [ ]:
food_sources = geopandas.read_file(food_sources_file)
bus_stops = geopandas.read_file(bus_stops_file)
buildings = geopandas.read_file(building_footprints_file)

In [ ]:
food_sources.shape

In [ ]:
bus_stops.shape

In [ ]:
buildings.shape

## Use OSM for GeoDataFrame index

In [ ]:
buildings.set_index("osm_id", inplace=True, verify_integrity=True)
bus_stops.set_index("osm_id", inplace=True, verify_integrity=True)
food_sources.set_index("osm_id", inplace=True, verify_integrity=True)

## Building centroids

In [ ]:
buildings["centroid"] = buildings["geometry"].centroid

In [ ]:
buildings.head()

In [ ]:
buildings.rename(
    columns={
        "geometry": "footprint",
        "centroid": "geometry"
    }, 
    inplace=True
)

In [ ]:
buildings.head()

In [ ]:
buildings.dtypes

## Create buffers

### Food sources

In [ ]:
buffer_distance_m = 1000

food_sources_buffered = food_sources.buffer(buffer_distance_m)

type(food_sources_buffered)

### Bus stops

In [ ]:
bus_stops.plot()

In [ ]:
%%time
buffer_distance_m = 250

bus_stops_buffered = bus_stops.buffer(buffer_distance_m)

bus_stops_buffered.plot()

## Dissolve buffers

### Food sources

In [ ]:
food_sources_catchment = food_sources_buffered.unary_union
food_sources_catchment

### Bus stops

In [ ]:
bus_stops_catchment = bus_stops_buffered.unary_union
bus_stops_catchment

## Access to amenities

In [ ]:
%%time
access_to_amenities = bus_stops_catchment.intersection(food_sources_catchment)
access_to_amenities

## Buildings within food catchment

In [ ]:
%%time
buildings["within_food_catchment"] = buildings.within(food_sources_catchment)

In [ ]:
%%time
buildings["access_to_amenities"] = buildings.within(access_to_amenities)

In [ ]:
buildings.head()

In [ ]:
buildings.plot(categorical=True, column="within_food_catchment",figsize=(20, 15), cmap='Spectral')

In [ ]:
buildings.plot(categorical=True, column="access_to_amenities",figsize=(20, 15), cmap='Spectral')

In [ ]:
buildings.head()

In [ ]:
buildings["access_to_amenities"].value_counts(normalize=True)